In [ ]:
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import datetime as dt
from lxml import html
import requests
from scipy.optimize import minimize
from nelson_siegel_svensson.calibrate import calibrate_nss_ols
import plotly.graph_objects as go


In [ ]:
# Set the headers for the HTTP request
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'} 

# Define a function to get risk-free rates
def get_risk_free():
    # URL of the treasury website from where we will scrape the data
    url = 'https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=2024'
    
    # Send a GET request to the treasury website
    page = requests.get(url, headers=headers)
    
    # Parse the HTML content of the page
    root = html.fromstring(page.content)
    
    # XPath to the element containing the rates
    rates_path = '//*[@id="block-hamilton-content"]/div/div/div[3]/table/tbody/tr[85]/td/text()'
    
    # Extract the rates using the XPath
    rates = root.xpath(rates_path)
    
    # Clean the rates by removing 'N/A' and converting to float
    clean_rates = np.array([float(rate.strip()) for rate in rates if rate.strip() != 'N/A' and rate.strip()]).astype(float)/100
    
    # Define the maturities
    maturities = np.array([1/12, 2/12, 3/12,4/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
    
    # Return the maturities and the corresponding rates
    return maturities, clean_rates

In [26]:
# Define the objective function for the Heston model calibration
def objective_function(params, options_df, spot_price, dividend_yield):
    # Unpack parameters: kappa, theta, sigma, rho, v0
    kappa, theta, sigma, rho, v0 = params

    # Initialize an empty list to store the errors
    errors = []

    # Get today's date
    today = ql.Date.todaysDate()

    # Create a handle for the spot price
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))

    # Create a flat forward yield term structure for the dividend yield
    dividend_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, dividend_yield, ql.Actual365Fixed()))

    # Iterate over each option in the options dataframe
    for index, option in options_df.iterrows():
        # Create a flat forward yield term structure for the risk-free rate
        r = ql.YieldTermStructureHandle(ql.FlatForward(today, option['rate'], ql.Actual365Fixed()))

        # Get the option's last price and strike price
        P = option['lastPrice']
        K = option['strike']

        # Create a QuantLib date for the option's expiration date
        ql_maturity_date = ql.Date(option['expiration'].day, option['expiration'].month, option['expiration'].year)

        # Define the payoff and exercise for a European call option
        payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)
        exercise = ql.EuropeanExercise(ql_maturity_date)

        # Create a QuantLib vanilla option with the defined payoff and exercise
        option_ql = ql.VanillaOption(payoff, exercise)

        # Define the Heston process and model, and set the pricing engine for the option
        heston_process = ql.HestonProcess(r, dividend_ts, spot_handle, v0, kappa, theta, sigma, rho)
        heston_model = ql.HestonModel(heston_process)
        engine = ql.AnalyticHestonEngine(heston_model)
        option_ql.setPricingEngine(engine)

        # Calculate the model price and the squared error
        model_price = option_ql.NPV()
        error = (model_price - P) ** 2

        # Append the error to the list of errors
        errors.append(error)

    # Sum up all the errors
    total_error = np.sum(errors)

    # Print the total error and the parameters
    print(total_error, params)

    # Return the total error
    return total_error

In [27]:
def fetch_option_data(ticker_symbol, option_type=ql.Option.Call):
    # Create a Ticker object for the given symbol
    ticker = yf.Ticker(ticker_symbol)
    
    # Get the closing price of the ticker for the latest trading day
    spot = ticker.history(period="1d")['Close'].iloc[-1]
    
    # Initialize an empty DataFrame to store options data
    options_df = pd.DataFrame()
    
    # Initialize a dictionary to store strikes by expiration
    strikes_by_expiration = {}

    # Iterate over each expiration date for the ticker's options
    for expiration in ticker.options:
        # Skip the expiration if it's after a certain date
        if pd.to_datetime(expiration) > pd.Timestamp('2025-06-20'):
            continue

        # Get the call or put options for the expiration date
        options = ticker.option_chain(expiration).calls if option_type == ql.Option.Call else ticker.option_chain(expiration).puts
        
        # Add the expiration date to the options DataFrame
        options["expiration"] = expiration
        
        # Append the options to the main DataFrame
        options_df = pd.concat([options_df, options], ignore_index=True)
        
        # Add the strikes to the dictionary
        strikes_by_expiration[expiration] = set(options['strike'])

    # Filter out options with low volume
    options_df = options_df[options_df['volume'] > 10]

    # Get the common strikes across all expiration dates
    common_strikes = set.intersection(*strikes_by_expiration.values())
    
    # Filter the options DataFrame to only include the common strikes
    options_df = options_df[options_df['strike'].isin(common_strikes)]
    
    # Convert the expiration date to a datetime object and calculate the maturity
    options_df['expiration'] = pd.to_datetime(options_df['expiration'])
    options_df['maturity'] = (options_df['expiration'] - pd.Timestamp('now')).dt.days / 365.25
    
    # Pivot the DataFrame to get the last price for each strike and maturity
    pivot_df = options_df.pivot_table(index='maturity', columns='strike', values='lastPrice', aggfunc='mean')
    
    # Melt the pivoted DataFrame back into a long format
    volSurfaceLong = pivot_df.melt(ignore_index=False, var_name='strike', value_name='lastPrice').reset_index()

    # Get the risk-free rates and maturities
    clean_rates, maturities = get_risk_free()
    
    # Calibrate the Nelson-Siegel-Svensson model
    curve_fit, status = calibrate_nss_ols(clean_rates, maturities)
    
    # Apply the calibrated model to get the risk-free rate for each maturity
    volSurfaceLong['rate'] = volSurfaceLong['maturity'].apply(lambda x: curve_fit.zero(x))

    # Map the maturity to the actual expiration date
    maturity_to_expiration = options_df[['maturity', 'expiration']].drop_duplicates().set_index('maturity')['expiration']
    volSurfaceLong['expiration'] = volSurfaceLong['maturity'].map(maturity_to_expiration)
    
    # Filter out options with a low last price
    volSurfaceLong = volSurfaceLong[volSurfaceLong['lastPrice'] > 0.1]

    # Return the DataFrame and the spot price
    return volSurfaceLong, spot

In [28]:
# Define a function to calibrate the Heston model parameters
def calibrate_heston_params(objective_function, options_df, spot, dividend_yield, bounds, initial_params):
    # Use the minimize function from scipy.optimize to find the parameters that minimize the objective function
    result = minimize(
        objective_function,  # The objective function to be minimized
        initial_params,  # The initial guess for the parameters
        method='L-BFGS-B',  # The optimization method
        tol=.00001,  # The tolerance for termination
        args=(options_df, spot, dividend_yield),  # Additional arguments passed to the objective function
        bounds=bounds,  # Bounds on the parameters
        options={'disp': True, 'maxiter': 10000}  # Options for the optimizer
    )
    # If the optimization was successful, return the optimized parameters
    if result.success:
        return result.x
    # If the optimization was not successful, raise an exception
    else:
        raise Exception("Calibration failed: " + result.message)

In [29]:
# Fetch option data for the S&P 500 index
options_df, spot = fetch_option_data("^SPX")

# Set the dividend yield for the options
dividend_yield = 0.0135  # Example dividend yield

# Define the bounds for the Heston model parameters
bounds = [
    (0.001, 6),    # kappa
    (0.001, 0.15), # theta
    (0.01, 2),     # sigma
    (-1, 1),       # rho
    (0.001, 0.5)   # v0
]

# Set the initial guesses for the Heston model parameters
initial_params = [3.32134696, 0.09693794,  0.29973543, -0.56363465,  0.3]  # Example initial guesses

# Try to calibrate the Heston model parameters
try:
    # Calibrate the Heston model parameters using the objective function, options data, spot price, dividend yield, bounds, and initial parameters
    calibrated_params = calibrate_heston_params(objective_function, options_df, spot, dividend_yield, bounds, initial_params)
    
    # Print the calibrated parameters
    print("Calibrated parameters:", calibrated_params)
    
# If the calibration fails, print an error message
except Exception as e:
    print("Calibration failed:", e)

Calibration failed: stdDev (-nan(ind)) must be non-negative


In [30]:
def calc_prices(spot, calibrated_params, dividend_yield, risk_free_rate, strike, maturity):
    # Unpack the calibrated parameters
    kappa, theta, sigma, rho, v0 = calibrated_params

    # Get today's date
    today = ql.Date.todaysDate()

    # Set the evaluation date to today
    ql.Settings.instance().evaluationDate = today

    # Define the day count convention
    day_count = ql.Actual365Fixed()

    # Define the calendar
    calendar = ql.UnitedStates(m=1)

    # Create a handle for the spot price
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot))

    # Create a flat forward yield term structure for the risk-free rate
    flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(risk_free_rate)), day_count))

    # Create a flat forward yield term structure for the dividend yield
    dividend_yield_handle = ql.YieldTermStructureHandle(ql.FlatForward(today, ql.QuoteHandle(ql.SimpleQuote(dividend_yield)), day_count))

    # Create a QuantLib date for the maturity
    ql_maturity_date = ql.Date(maturity.day, maturity.month, maturity.year)

    # Define the payoff and exercise for a European call option
    payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike)
    exercise = ql.EuropeanExercise(ql_maturity_date)

    # Create a QuantLib vanilla option with the defined payoff and exercise
    option_ql = ql.VanillaOption(payoff, exercise)

    # Define the Heston process and model, and set the pricing engine for the option
    heston_process = ql.HestonProcess(flat_ts, dividend_yield_handle, spot_handle, v0, kappa, theta, sigma, rho)
    heston_model = ql.HestonModel(heston_process)
    engine = ql.AnalyticHestonEngine(heston_model)
    option_ql.setPricingEngine(engine)

    # Calculate the model price
    model_price = option_ql.NPV()

    # Return the model price
    return model_price

In [31]:
from datetime import datetime

def fetch_data(ticker_symbol, option_type=ql.Option.Call, max_maturity_years=1, price_range_percentage=20):
    # Create a Ticker object for the given symbol
    ticker = yf.Ticker(ticker_symbol)

    # Get the closing price of the ticker for the latest trading day
    spot = ticker.history(period="1d")['Close'].iloc[-1]

    # Initialize an empty DataFrame to store options data
    options_df = pd.DataFrame()

    # Set maximum maturity date
    max_expiration_date = datetime.now() + pd.DateOffset(years=max_maturity_years)

    # Set the range of strike prices to consider
    lower_price_limit = spot * (1 - price_range_percentage / 100)
    upper_price_limit = spot * (1 + price_range_percentage / 100)

    # Iterate over each expiration date for the ticker's options
    for expiration in ticker.options:
        expiration_date = pd.to_datetime(expiration)

        # Skip the expiration if it's after a certain date or beyond the maximum maturity date
        if expiration_date > pd.Timestamp('2025-10-16') or expiration_date > max_expiration_date:
            continue

        # Get the call or put options for the expiration date
        options = ticker.option_chain(expiration).calls if option_type == ql.Option.Call else ticker.option_chain(expiration).puts

        # Add the expiration date to the options DataFrame
        options["expiration"] = expiration_date

        # Append the options to the main DataFrame
        options_df = pd.concat([options_df, options], ignore_index=True)

    # Convert the expiration date to a datetime object and calculate the maturity
    options_df['expiration'] = pd.to_datetime(options_df['expiration'])
    options_df['maturity'] = (options_df['expiration'] - pd.Timestamp('now')).dt.days / 365.25

    # Filter options based on strike price range
    options_df = options_df[(options_df['strike'] >= lower_price_limit) & (options_df['strike'] <= upper_price_limit)]

    # Ensure no NaN in 'lastPrice'
    options_df.dropna(subset=['lastPrice'], inplace=True)

    # Pivot the DataFrame to get the last price for each strike and maturity
    pivot_df = options_df.pivot_table(index='maturity', columns='strike', values='lastPrice', aggfunc='mean')

    # Melt the pivoted DataFrame back into a long format
    volSurfaceLong = pivot_df.melt(ignore_index=False, var_name='strike', value_name='lastPrice').reset_index()

    # Get the risk-free rates and maturities
    clean_rates, maturities = get_risk_free()

    # Calibrate the Nelson-Siegel-Svensson model
    curve_fit, status = calibrate_nss_ols(clean_rates, maturities)

    # Apply the calibrated model to get the risk-free rate for each maturity
    volSurfaceLong['rate'] = volSurfaceLong['maturity'].apply(lambda x: curve_fit.zero(x))

    # Map maturity to actual expiration dates
    maturity_to_expiration = options_df[['maturity', 'expiration']].drop_duplicates().set_index('maturity')['expiration']
    volSurfaceLong['expiration'] = volSurfaceLong['maturity'].map(maturity_to_expiration)

    # Return the DataFrame and the spot price
    return volSurfaceLong, spot

# Fetch option data for the S&P 500 index
volSurfaceLong, spot = fetch_data('^SPX')

In [ ]:
# Fetch option data for the S&P 500 index
volSurfaceLong, spot = fetch_data("^SPX")

# Drop any rows with NaN in 'lastPrice' column
volSurfaceLong = volSurfaceLong.dropna(subset=['lastPrice'])

# Initialize an empty list to store predicted prices
predicted_prices = []

# Iterate over each option in the DataFrame
for index, option in volSurfaceLong.iterrows():
    # Extract the risk-free rate, strike price, maturity date, and actual price for the option
    risk_free_rate = option['rate']
    strike = option['strike']
    maturity = option['expiration']
    dividend_yield = 0.0136
    actual_price = option['lastPrice']

    # Calculate the predicted price using the Heston model
    pred_price = calc_prices(spot, calibrated_params, dividend_yield, risk_free_rate, strike, maturity)

    # Append the predicted price to the list
    predicted_prices.append(pred_price)

    # Print the strike price, maturity date, actual price, and predicted price
    print(f"Strike: {strike}, Maturity: {maturity}, Actual Price: {actual_price}, Predicted Price: {pred_price}")

# Add the predicted prices as a new column in the DataFrame
volSurfaceLong['Predicted Price'] = predicted_prices

# Convert 'Maturity' from datetime to numerical days to maturity
volSurfaceLong['Days to Maturity'] = (volSurfaceLong['expiration'] - pd.Timestamp('now')).dt.days

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Actual Prices
fig.add_trace(go.Scatter3d(
    x=volSurfaceLong['Days to Maturity'],
    y=volSurfaceLong['strike'],
    z=volSurfaceLong['lastPrice'],
    mode='markers',
    marker=dict(size=4, color='blue'),  # Increase marker size here
    name='Actual Price'
))

# Predicted Prices
fig.add_trace(go.Scatter3d(
    x=volSurfaceLong['Days to Maturity'],
    y=volSurfaceLong['strike'],
    z=volSurfaceLong['Predicted Price'],
    mode='markers',
    marker=dict(size=4, color='red'),  # Increase marker size here
    name='Predicted Price'
))

# Layout settings
fig.update_layout(
    title='Actual vs Predicted Option Prices',
    scene=dict(
        xaxis_title='Days to Maturity',
        yaxis_title='Strike Price',
        zaxis_title='Option Price'
    ),
    legend_title="Legend",
    width=1500,  # Set the width of the figure
    height=1200,  # Set the height of the figure
    autosize=False  # Disable autosizing to use the specified width and height
)

fig.show()


In [ ]:
errors = volSurfaceLong['Predicted Price'] - volSurfaceLong['lastPrice']
mae = np.mean(np.abs(errors))
mse = np.mean(np.square(errors))
rmse = np.sqrt(mse)

# Calculating correlation coefficient
correlation_matrix = np.corrcoef(volSurfaceLong['lastPrice'], volSurfaceLong['Predicted Price'])
correlation = correlation_matrix[0, 1]

# Displaying error metrics and correlation
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Correlation Coefficient between Actual and Predicted Prices:", correlation)

# Plotting residuals
plt.figure(figsize=(10, 5))
plt.scatter(volSurfaceLong['lastPrice'], errors, color='red')
plt.axhline(y=0, color='black', linestyle='--')
plt.xlabel('Actual Prices')
plt.ylabel('Residuals (Errors)')
plt.title('Plot of Residuals')
plt.grid(True)
plt.show()
